In [0]:
from pyspark.sql.functions import col, avg, count, when
spark.sql("CREATE DATABASE IF NOT EXISTS hotel_monitoring")
df = spark.table("hotel_silver.bookings_clean")

rules = {
    "adr_negative": df.filter(col("adr") <= 0).count(),
    "null_country": df.filter(col("country").isNull()).count(),
    "null_customer_type": df.filter(col("customer_type").isNull()).count()
}

rules

from pyspark.sql import Row
import datetime

rows = [
    Row(rule=k, failures=v, run_date=datetime.date.today())
    for k, v in rules.items()
]

spark.createDataFrame(rows) \
  .write.format("delta") \
  .mode("append") \
  .saveAsTable("hotel_monitoring.data_quality_results")

